<a href="https://colab.research.google.com/github/datascience-uniandes/data-analysis-tutorial/blob/master/fifa/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis (EDA)

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
Dataset: FIFA
  
Last update: August, 2024

In [4]:
pip install --upgrade ydata-profiling

  Using cached ydata_profiling-4.9.0-py2.py3-none-any.whl (356 kB)
  Using cached statsmodels-0.14.2-cp39-cp39-win_amd64.whl (9.9 MB)
  Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
  Using cached numba-0.60.0-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached wordcloud-1.9.3-cp39-cp39-win_amd64.whl (300 kB)
  Using cached typeguard-4.3.0-py3-none-any.whl (35 kB)
  Using cached phik-0.12.4-cp39-cp39-win_amd64.whl (666 kB)
  Using cached dacite-1.8.1-py3-none-any.whl (14 kB)
  Using cached ImageHash-4.3.1-py2.py3-none-any.whl (296 kB)
  Using cached htmlmin-0.1.12-py3-none-any.whl
  Using cached llvmlite-0.43.0-cp39-cp39-win_amd64.whl (28.1 MB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.37.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
# Importing libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from ydata_profiling import ProfileReport

C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


AttributeError: module 'matplotlib.cm' has no attribute 'register_cmap'

In [4]:
# Panadas configuration to extend the number of rows and columns to visualize by default
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [5]:
# Suppress scientific notation
pd.options.display.float_format = '{:.2f}'.format

### Load the data

In [6]:
# Loading the CSV file as a dataframe
fifa_df = pd.read_csv('./data/fifa.csv')

C:\Users\Usuario\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (22,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Showing the dataframe dimensions
fifa_df.shape

In [ ]:
# Showing the column data types
fifa_df.dtypes

In [ ]:
# Showing the top 5 rows
fifa_df.head()

### Transform some columns

In [ ]:
# Analyzing unique values for columns Value and Wage
set(''.join(fifa_df['Value'].values + fifa_df['Wage'].values))

In [ ]:
# Transforming some columns to float based on thousand and million symbols

def money_str_to_float(x):
    x = x.replace('€', '')           # Remove the euro symbol
    if 'M' in x:                     # If the value has the million symbol 
        x = x.replace('M', '')       # Remove the symbol
        return float(x) * 1000000    # Cast the value to float and multiply it by 1'000.000
    elif 'K' in x:                   # If value has the thousand symbol
        x = x.replace('K', '')       # # Remove the symbol
        return float(x) * 1000       # Cast the value to float and multiply it by 1.000
    else:                            # If neither symbol is found
        return float(x)              # Only cast the value to float

fifa_df['Value'] = fifa_df['Value'].apply(money_str_to_float)
fifa_df['Wage'] = fifa_df['Wage'].apply(money_str_to_float)

In [ ]:
fifa_df.head()

### Explore the *Value* column

In [ ]:
# Getting the mean Value for all players
fifa_df['Value'].mean()

In [ ]:
# Getting the median value for all players
fifa_df['Value'].median()

In [ ]:
# Getting the player with the highest value
max_value = fifa_df['Value'].max()
fifa_df.loc[fifa_df['Value'] == max_value]

In [ ]:
# Getting the main statistics for the Value column
fifa_df['Value'].describe()

<p style="color: red;">Modify the code above to show percentiles 95 and 99</p>

In [ ]:
# Computing skewness and kurtosis
print("Skewness:", fifa_df['Value'].skew())
print("Kurtosis:", fifa_df['Value'].kurtosis())

In [ ]:
# Plotting the histogram for the column
fifa_df['Value'].hist()
plt.show()

In [ ]:
# Defining the number of bins and for better visualization
plt.figure()
fifa_df['Value'].hist(bins=50)
plt.show()

In [ ]:
# Plotting distribution but using a boxplot
plt.figure(figsize=(18, 3))
plt.boxplot(fifa_df['Value'], showmeans=True, vert=False)
plt.show()

In [ ]:
# Which are the most valuable players?
outlier_threshold = fifa_df['Value'].mean() + fifa_df['Value'].std() * 15
fifa_df[fifa_df['Value'] > outlier_threshold]

In [ ]:
# Using the method based on quartiles to select outliers
iqr = (fifa_df['Value'].quantile(.75) - fifa_df['Value'].quantile(.25))
outlier_threshold = fifa_df['Value'].quantile(.75) + iqr * 45
fifa_df[fifa_df['Value'] > outlier_threshold]

### Explore the *Preferred Positions* column

In [ ]:
fifa_df['Preferred Positions'].unique()

In [ ]:
# How many preferred positions the players have?

def count_pref_positions(x):
    positions = x.strip().split(' ')    # Split the text by space
    return len(positions)               # Count the number of positions and return the value

fifa_df['Preferred Positions'].apply(count_pref_positions).value_counts(dropna=False, normalize=True)

In [ ]:
# Selecting the preferred position with the highest score

def get_pref_position(row):
    pref_positions = row['Preferred Positions'].strip().split(' ')    # Split the text by space
    if pref_positions[0] == "GK":                                     # Handle the special case for GKs
        return "GK"
    scores = np.array([row[p] for p in pref_positions])               # Get the the scores for each preferred position
    i_max = np.argmax(scores)                                         # Get the index for the highest score
    return pref_positions[i_max]                                      # Locate the position using its index and return it

fifa_df['Preferred Position'] = fifa_df.apply(get_pref_position, axis=1)

In [ ]:
# Calculating the absolute frecuency of the preferred position
pref_positions_abs_frec = fifa_df['Preferred Position'].value_counts(dropna=False)
pref_positions_abs_frec

In [ ]:
# Plotting preferred position frecuency
# WARNING: line chart assumes continuity. It is a better choice when dealing with continuous variables (e.g. time series)
pref_positions_abs_frec.plot()
plt.show()

<p style="color: red;">Use a bar chart to better visualize the preferred position frecuency</p>

### Explore the relationship between *Age* and *Value*

In [ ]:
# Plotting a scatter plot to visualize two quantitative variables
plt.figure(figsize=(13, 7))
plt.scatter(fifa_df['Age'], fifa_df['Value'], alpha=0.5)
plt.title('Age vs. Value')
plt.xlabel('Age')
plt.ylabel('Value')
plt.show()

In [ ]:
# What is the (linear) correlation coefficient?
fifa_df[['Age', 'Value']].corr()

In [ ]:
# What is the (monotonic) correlation coefficient?
fifa_df[['Age', 'Value']].corr('spearman')

### Explore the relationship between *Overall* and *Value*

In [ ]:
plt.figure(figsize=(13, 7))
plt.scatter(fifa_df['Overall'], fifa_df['Value'], alpha=0.5, color='orange')
plt.title('Overall vs. Value')
plt.xlabel('Overall')
plt.ylabel('Value')
plt.show()

In [ ]:
# What is the (linear) correlation coefficient?
fifa_df[['Overall', 'Value']].corr()

In [ ]:
# What is the (monotonic) correlation coefficient?
fifa_df[['Overall', 'Value']].corr('spearman')

<p style="color: red;">Why both methods give different correlation coefficients for the same pair of variables?</p>

### Compare by *Club*

In [ ]:
# Plotting Wage by club
plt.figure(figsize=(13, 10))
sns.boxplot(
    data=fifa_df.loc[fifa_df['Club'].isin(['Real Madrid CF', 'FC Barcelona'])],
    x='Club',
    y='Wage',
    showmeans=True
)
plt.show()

In [ ]:
# Plotting Wage by club
plt.figure(figsize=(13, 10))
sns.boxplot(
    data=fifa_df.loc[fifa_df['Club'].isin(['Independiente Santa Fe', 'CD Los Millionarios Bogota'])],
    x='Club',
    y='Wage',
    showmeans=True
)
plt.show()

<p style="color: red;">What are the main origins (Nationality) of the players in the top 3 teams with the most expensive squads (Value)? Hint: You need to group the data first.</p>

### Profiling the data

In [ ]:
profile = ProfileReport(fifa_df)

In [ ]:
# profile.to_notebook_iframe()

In [ ]:
# profile.to_file('report.html')

### Exporting clean date

In [ ]:
fifa_df.to_csv('./data/fifa_clean.csv', index=False)